In [ ]:
!pip install opencv-python numpy matplotlib scikit-learn imbalanced-learn keras 
!pip install tensorflow

In [1]:
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.discriminant_analysis import StandardScaler
from sklearn.metrics import accuracy_score
from scipy.stats import loguniform
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA


In [2]:
folder_path = 'extract_lfw'
df_images = []
labels = []  # List to store image labels
for label, person in enumerate(os.listdir(folder_path)):
    if os.path.isdir(os.path.join(folder_path, person)):
        image_files = [filename for filename in os.listdir(os.path.join(folder_path, person)) if filename.endswith('.jpg')]
        
        # Check if there are images in the subfolder
        if image_files:
            for filename in image_files:
                img_path = os.path.join(folder_path, person, filename)
                img = cv2.imread(img_path,cv2.IMREAD_GRAYSCALE)
                img = cv2.normalize(img, None, 0, 1.0,cv2.NORM_MINMAX, dtype=cv2.CV_32F)
                img = img.flatten()
                df_images.append(img)
                labels.append(label)  # Assigning label based on folder/class
        else:
            print(f"Skipping '{person}' due to no images in the subfolder.")
df_images = np.array(df_images)
labels = np.array(labels)


In [3]:
from sklearn.utils import shuffle

# Assuming 'X' is your data and 'y' is the corresponding labels
df_images, labels = shuffle(df_images, labels , random_state=42)

In [4]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_images, labels, test_size=0.3, random_state=42)

# Setting df_images and labels to 0 to deallocate memory
df_images = 0
labels = 0

# Printing the shapes of the training and testing sets
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)


(2115, 62500) (2115,)
(907, 62500) (907,)


In [5]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

In [6]:
# Apply SMOTE to the training data
smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
print(X_resampled.shape)
print(y_resampled.shape)

(23374, 62500)
(23374,)


In [9]:
# Assuming df_images, X_train, and X_test are defined previously

# Apply PCA to the scaled training data
pca = PCA(n_components=150,svd_solver="arpack",whiten=True)
X_train_pca = pca.fit_transform(X_final)

# Apply the same transformation to the testing data
X_test_pca = pca.transform(X_test)



# Plot explained variance ratio
cumulative_var_ratio = np.cumsum(pca.explained_variance_ratio_)
plt.plot(cumulative_var_ratio)
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.title('Cumulative Explained Variance Ratio')
plt.show()

# Print the shape of transformed data after PCA
print("X_train_pca shape:", X_train_pca.shape)
print("X_test_pca shape:", X_test_pca.shape)

MemoryError: Unable to allocate 10.9 GiB for an array with shape (46748, 62500) and data type float32

In [ ]:
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap='viridis', label='Training Data')

plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('Data Points in PCA Space')
plt.legend()
plt.show()

In [ ]:

Svm = SVC(C= 1.4872681066575715, gamma= 0.003293320249656128, kernel= "rbf", class_weight="balanced")
Svm.fit(X_train_pca, y_resampled)


y_pred =Svm.predict(X_test_pca)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")


In [ ]:
param_grid = {
    "C": loguniform(1e-1, 1e3),  # Adjusted range for 'C'
    "gamma": loguniform(1e-5, 1e-1),  # Adjusted range for 'gamma'
    'kernel': ['rbf', 'poly', 'sigmoid','linear']
}

# Use a scoring metric, such as accuracy
clf = RandomizedSearchCV(
    SVC(class_weight="balanced"), param_grid, cv=10, scoring='accuracy', n_iter=100
)

clf.fit(X_test_pca, y_resampled)
best_params = clf.best_params_
print("Best Parameters:", best_params)

y_pred = clf.predict(X_test_pca)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
